# 🎴 MTG Card Recognition System

**Approach:** Perceptual Hashing
**Dataset:** Scryfall API
**Max Cards:** 10000

This notebook implements a Magic: The Gathering card recognition system that can identify cards regardless of language or artwork variant.

## 1. Setup & Dependencies

In [1]:
# Install required packages (uncomment if needed)
# !pip install opencv-python pillow requests tqdm imagehash

import numpy as np
import cv2
from PIL import Image
import requests
import json
from pathlib import Path
from tqdm import tqdm
import time
import imagehash
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## 2. Data Collection

In [2]:
def fetch_scryfall_cards(max_cards=10000):
    """
    Fetch card data from Scryfall API.
    Returns a list of card objects with image URLs and metadata.
    """
    print(f'Fetching up to {max_cards} cards from Scryfall...')
    
    cards = []
    url = 'https://api.scryfall.com/cards/search?q=game:paper'
    
    while url and len(cards) < max_cards:
        response = requests.get(url)
        if response.status_code != 200:
            print(f'Error: {response.status_code}')
            break
            
        data = response.json()
        
        for card in data['data']:
            if 'image_uris' in card:
                cards.append({
                    'id': card['id'],
                    'name': card['name'],
                    'image_url': card['image_uris']['normal'],
                    'set': card.get('set', 'unknown'),
                    'collector_number': card.get('collector_number', '0')
                })
                
                if len(cards) >= max_cards:
                    break
        
        url = data.get('next_page')
        time.sleep(0.1)  # Respect API rate limits
    
    print(f'Fetched {len(cards)} cards')
    return cards

# Fetch the cards
cards_data = fetch_scryfall_cards()
print(f'Total cards collected: {len(cards_data)}')

Fetching up to 10000 cards from Scryfall...
Fetched 10000 cards
Total cards collected: 10000


## 3. Card Detection (OpenCV)

In [3]:
def detect_card(image):
    """
    Detect and extract a card from an image.
    Returns the cropped and perspective-corrected card image.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Threshold
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    # Find largest rectangular contour
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Approximate to polygon
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    # If we found a quadrilateral
    if len(approx) == 4:
        # Get card dimensions (standard MTG card ratio: 2.5:3.5)
        width, height = 250, 350
        
        # Destination points
        dst_pts = np.array([
            [0, 0],
            [width - 1, 0],
            [width - 1, height - 1],
            [0, height - 1]
        ], dtype='float32')
        
        # Source points
        src_pts = approx.reshape(4, 2).astype('float32')
        
        # Calculate perspective transform
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        
        # Apply transform
        warped = cv2.warpPerspective(image, M, (width, height))
        
        return warped
    
    return None

print('Card detection function ready')

Card detection function ready


## 4. Feature Extraction & Recognition

In [4]:
def build_hash_database(cards_data):
    """
    Build a database of perceptual hashes for all cards.
    """
    hash_db = {}
    
    print('Building hash database...')
    for card in tqdm(cards_data):
        try:
            # Download image
            response = requests.get(card['image_url'])
            img = Image.open(requests.get(card['image_url'], stream=True).raw)
            
            # Calculate perceptual hash
            img_hash = imagehash.phash(img)
            
            # Store in database
            hash_db[str(img_hash)] = card
            
        except Exception as e:
            print(f"Error processing {card['name']}: {e}")
            continue
    
    return hash_db

def recognize_card(query_image, hash_db, threshold=5):
    """
    Recognize a card using perceptual hashing.
    Returns the best match if distance < threshold.
    """
    # Convert to PIL Image if needed
    if isinstance(query_image, np.ndarray):
        query_image = Image.fromarray(cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB))
    
    # Calculate hash
    query_hash = imagehash.phash(query_image)
    
    # Find best match
    best_match = None
    best_distance = float('inf')
    
    for hash_str, card_data in hash_db.items():
        distance = query_hash - imagehash.hex_to_hash(hash_str)
        
        if distance < best_distance:
            best_distance = distance
            best_match = card_data
    
    if best_distance <= threshold:
        return best_match, best_distance
    
    return None, best_distance

# Build the database
hash_database = build_hash_database(cards_data)
print(f'Hash database built with {len(hash_database)} cards')

Building hash database...


100%|██████████| 10000/10000 [35:21<00:00,  4.71it/s] 

Hash database built with 9997 cards


## 5. Complete Recognition Pipeline

In [5]:
def identify_mtg_card(image_path):
    """
    Complete pipeline: load image -> detect card -> recognize.
    """
    # Load image
    image = cv2.imread(image_path)
    
    if image is None:
        return None, 'Failed to load image'
    
    # Detect card
    detected_card = detect_card(image)
    
    if detected_card is None:
        print('No card detected, using full image')
        detected_card = image
    
    # Recognize card
    result, distance = recognize_card(detected_card, hash_database)
    
    if result:
        return result, f'Match found with distance: {distance}'
    else:
        return None, f'No match found (best distance: {distance})'

print('Complete pipeline ready!')
print('Usage: result, info = identify_mtg_card("path/to/card/image.jpg")')

Complete pipeline ready!
Usage: result, info = identify_mtg_card("path/to/card/image.jpg")


## 6. Testing & Examples

In [10]:
test_image_path = 'img/atraxa.jpg'

# Uncomment to test:
result, info = identify_mtg_card(test_image_path)
if result:
    print(f"Card identified: {result['name']}")
    print(f"Set: {result['set']}")
    print(info)
else:
    print(f"Identification failed: {info}")

Identification failed: No match found (best distance: 14)


## 7. Visualization

In [7]:
def visualize_result(image_path, result, info):
    """
    Visualize the recognition result.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Original image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[0].imshow(img_rgb)
    axes[0].set_title('Input Image')
    axes[0].axis('off')
    
    # Matched card
    if result:
        matched_img = Image.open(requests.get(result['image_url'], stream=True).raw)
        axes[1].imshow(matched_img)
        axes[1].set_title(f"Matched: {result['name']}\n{info}")
    else:
        axes[1].text(0.5, 0.5, 'No match found', ha='center', va='center')
        axes[1].set_title(info)
    
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()

print('Visualization function ready')

Visualization function ready


## 8. Evaluation Metrics

In [8]:
def evaluate_accuracy(test_images, ground_truth):
    """
    Evaluate the accuracy of the recognition system.
    """
    correct = 0
    total = len(test_images)
    
    for img_path, true_card_id in zip(test_images, ground_truth):
        result, _ = identify_mtg_card(img_path)
        
        if result and result['id'] == true_card_id:
            correct += 1
    
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2%} ({correct}/{total})')
    return accuracy

print('Evaluation function ready')

Evaluation function ready


## 🎯 Next Steps

### To use this notebook:

1. **Run all cells** to initialize the system
2. **Wait for database building** (may take 10-30 minutes depending on card count)
3. **Test with your images** using `identify_mtg_card(image_path)`

### Improvements to consider:

- Add caching to save the hash database locally
- Implement batch processing for multiple cards
- Add support for double-faced cards
- Create a web interface using Gradio or Streamlit
- Fine-tune detection for specific lighting conditions

### Performance notes:

- **Perceptual Hash**: ~50-100ms per card, works well with variations

Happy card hunting! 🎴✨